# Reviewing Model Generated Answers

In this notebook we'll load and review answers generated by both the base and fine-tuned models. We only have enough GPU memory to load one of these models at a time so we'll need to clear memory between different model runs!

# Set Base Directories

In [1]:
%env TOKENIZERS_PARALLELISM=false

DATA_DIRECTORY = "../data"
MODEL_DIRECTORY = "../model"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

env: TOKENIZERS_PARALLELISM=false


# Load Train/Test Data Frame

In [2]:
import pandas as pd

path = f"{DATA_DIRECTORY}/train-test-df.csv"
df = pd.read_csv(path, na_filter=False)
print(f"Loaded {df.shape[0]:,d} Train/Test records.")
#print(df.fold.value_counts())
df.sample(n=1)

Loaded 1,050 Train/Test records.


,fold,excerpt,question,answer,hashID
702,2,"To the untrained eye, modern drones that can p...",What factors contribute to the mistaken identi...,The mistaken identification of drones as UFOs ...,a15b9e69b53e2e1f92a7237462e842a2


# Load Evaluation Prompt Prefix

In [3]:
path = f"{DATA_DIRECTORY}/level-1-prompt-prefix.txt"
with open(path) as ifp: prefix = ifp.read()

print(prefix)

Carefully read the excerpt below and then provide a clear concise answer to the follow-up question.


# Load Model, Tokenizer and Generator

In [4]:
import torch
from utilities import display_sample, load_base_model, load_lora

# Clear memory between different model runs!

In [5]:
tokenizer, model, generator = None, None, None
torch.cuda.empty_cache()
!gpustat

ip-172-25-5-124      Thu Oct  5 18:53:23 2023  535.54.03
[0] NVIDIA A10G      | 33'C,   0 % |   309 / 23028 MB |


## Base Model

In [ ]:
tokenizer, model, generator = load_base_model(model_name)

## Fine-tuned Model

In [6]:
fold = 10
directory = f"{MODEL_DIRECTORY}/Llama-2-7b-qa-{fold:02d}"
tokenizer, model, generator = load_lora(MODEL_NAME, directory)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Sample Predictions

In [9]:
x = df.sample(n=1).iloc[0]
display_sample(x, prefix, generator, max_new_tokens=256)

INSTRUCTIONS:
+------------------------------------------------------------------------------------------------------+
| Carefully read the excerpt below and then provide a clear concise answer to the follow-up question.  |
+------------------------------------------------------------------------------------------------------+

EXCERPT:
+------------------------------------------------------------------------------------------------------+
| The air defense system they wanted to hit is no longer there.  UNIDENTIFIED MALE: (SPEAKING IN       |
| FOREIGN LANGUAGE). BELL: Home for tonight is a field about 15 kilometers north of the frontline,     |
| using only red lights to avoid detection, they've got a bird's eye view of the battle below and      |
| what's happening beyond.  UNIDENTIFIED MALE: We are hunting for them for some time, we have some     |
| results. We know where they're hiding. We know where they are moving. So it's a question about       |
| trying to justify them.  BELL

In [ ]:
!nvidia-smi
